In [1]:
import os

import pandas

import dask.dataframe as dd

from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()

from shared_code.utility.storage.table import TableAdapter

In [2]:
from tqdm.dask import TqdmCallback

cb = TqdmCallback(desc="global")
cb.register()

from tqdm import tqdm_notebook, tqdm

tqdm_notebook().pandas()
tqdm.pandas(desc="global")

C:\Users\AJ Stangl\AppData\Local\Temp\ipykernel_20508\1542666549.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

In [3]:
from tqdm import tqdm

class InnerProgressBar(tqdm):
	def __init__(self, total, desc):
		super().__init__(desc=desc)
		self.total = total
		self.current = 0

	def update_to(self):
		self.update(self.current)

In [4]:
table_name = "training"

table_adapter: TableAdapter = TableAdapter()

data_path = "/data/parquet/"

parquet_raw_data_path = data_path + "raw_data.parquet"

parquet_process_data_path = data_path + "processed_data.parquet"

In [5]:
os.makedirs(data_path, exist_ok=True)

print(f"Data Path:\t{data_path}\n\nExists:\t{os.path.exists(data_path)}")

Data Path:	/data/parquet/

Exists:	True


In [6]:
pandas_df = None
if os.path.exists("foo"):
	print("Loading from parquet")
	pandas_df = pandas.read_parquet(parquet_raw_data_path)


else:
	print("Loading from table")
	raw_data = table_adapter.get_all_entities(table_name)
	pandas_df = pandas.DataFrame(list(raw_data))
	print(f"Saving to parquet {parquet_process_data_path}")
	pandas_df.to_parquet(parquet_raw_data_path)

Loading from parquet


In [7]:
print(f"Initial Dataframe Shape:\t{pandas_df.shape}")

display(pandas_df)

Initial Dataframe Shape:	(8600, 19)


,PartitionKey,RowKey,author,caption,curated,exists,exits,flair,hash,id,image,image_name,permalink,small_image,subreddit,text,updated_caption,updated_path,url
0,training,1000cej,OtterlyFoxy,a city street filled with lots of tall buildings,False,True,True,NaN,7a8d96e378c15c8ab8440ac311f12c11,1000cej,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,CityPorn,New York in the fog,a city street filled with lots of tall buildings,D:\data\images\CityPorn\4emw5uldib9a1.jpg,https://i.redd.it/4emw5uldib9a1.jpg
1,training,1000d16,princessxo699,a woman sitting on a table holding a pink flower,False,True,True,Outfit of the Day,9951b4f82caeb8ba2bd9f79f8d422450,1000d16,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,GgFEagO.jpg,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,SFWNextDoorGirls,Thoughts about my NYE party outfit?,a woman sitting on a table holding a pink flower,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,https://i.imgur.com/GgFEagO.jpg
2,training,1000fg0,BlkBrd1312,a woman in a bikini with a cell phone in her hand,False,True,True,HotGirlNextDoor,be8dd55e34216bec1e15e03fa296eacc,1000fg0,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,nwa7hts2jb9a1.jpg,/r/HotGirlNextDoor/comments/1000fg0/iktr/,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,HotGirlNextDoor,(IKTR),a woman in a bikini with a cell phone in her hand,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,https://i.redd.it/nwa7hts2jb9a1.jpg
3,training,1000glf,toolate_sharkbait,a beautiful young woman in a black dress posin...,False,True,True,Female (18+),e554c1ed7ffa2740436ac082068b2824,1000glf,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,/r/AmIhotAF/comments/1000glf/just_looking_for_...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,AmIhotAF,Just looking for entertainment,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,https://i.redd.it/4xyb1vgbjb9a1.jpg
4,training,1000j1n,trent8051,a collage of photos showing a man in a suit an...,False,True,None,None,1dec3dabb5e46cde01855d06089c287a,1000j1n,D:\data\images\greentext\3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,/r/greentext/comments/1000j1n/anon_wants_elon_...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,greentext,Anon wants Elon cut,a collage of photos showing a man in a suit an...,None,https://i.redd.it/3mewbe0wjb9a1.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8595,training,zzyerp,Juliadomaina,A girl in a red jacket is sitting on a red horse.,False,True,True,Female (18+),9279353b1761d62e7f8ca587a4146abf,zzyerp,D:\data\images\AmIhotAF\hgktc1on2b9a1.jpg,hgktc1on2b9a1.jpg,/r/AmIhotAF/comments/zzyerp/good_evening_every...,D:\data\images\AmIhotAF\thumbnail\hgktc1on2b9a...,AmIhotAF,Good evening everyone ❤️,a woman in a bikini with a cell phone in her hand,D:\data\images\AmIhotAF\hgktc1on2b9a1.jpg,https://i.redd.it/hgktc1on2b9a1.jpg
8596,training,zzzeoi,ComprehendApprehend,A man wearing a black shirt and black pants is...,False,True,None,None,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,D:\data\images\greentext\bgho6WK.jpg,bgho6WK.jpg,/r/greentext/comments/zzzeoi/anon_does_a_littl...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,greentext,Anon does a little trolling,a series of photos showing a man in a suit,None,https://i.imgur.com/bgho6WK.jpg
8597,training,zzzg13,AdConsistent3184,A little girl in a pink dress is sitting on a ...,False,False,False,NaN,ba89401054b09d3ff2b608705f8a7110,zzzg13,D:\data\images\SFWRedheads\3rv0ql34bb9a1.jpg,3rv0ql34bb9a1.jpg,/r/SFWRedheads/comments/zzzg13/cute_but_deadly/,,SFWRedheads,Cute but Deadly 😋,NaN,D:\data\images\SFWRedheads\3rv0ql34bb9a1.jpg,https://i.redd.it/3rv0ql34bb9a1.jpg
8598,training,zzzlyl,Particular-Chip2355,Error in captioning image,False,False,True,NaN,3cac9b29f27d5fc26ddcec4e1641649f,zzzlyl,D:\data\images\CityPorn\Minquan_Bridge20

In [8]:
print("Normalizing Dataframe For Processing")

subreddit = pandas_df["subreddit"]
image = pandas_df["image_name"]
text = pandas_df["text"]
hash_value = pandas_df["hash"]
original_caption = pandas_df["caption"]
thumbnail_caption = pandas_df["updated_caption"]
comment_id = pandas_df["id"]
all_normalized_frame = pandas.DataFrame(
	{
		"subreddit": subreddit,
		"file_name": image,
		"text": text,
		"original_caption": original_caption,
		"thumbnail_path": pandas_df.apply(lambda x: f"D:\\data\\images\\{x['subreddit']}\\thumbnail\\{x['image_name']}",
										  axis=1),
		"thumbnail_caption": thumbnail_caption,
		"thumbnail_exists": pandas_df.apply(
			lambda x: os.path.exists(f"D:\\data\\images\\{x['subreddit']}\\thumbnail\\{x['image_name']}"), axis=1),
		"original_image": pandas_df.apply(lambda x: f"D:\\data\\images\\{x['subreddit']}\\{x['image_name']}", axis=1),
		"original_image_exists": pandas_df.apply(
			lambda x: os.path.exists(f"D:\\data\\images\\{x['subreddit']}\\thumbnail\\{x['image_name']}"), axis=1),
		"hash": hash_value,
		"id": comment_id
	}
)
print(f"Normalized Dataframe Shape:\t{all_normalized_frame.shape}")
print(f"Saving to parquet {parquet_process_data_path}")
all_normalized_frame.to_parquet(parquet_process_data_path)

Normalizing Dataframe For Processing
Normalized Dataframe Shape:	(8600, 11)
Saving to parquet /data/parquet/processed_data.parquet


In [9]:
print(f"Reading from parquet {parquet_process_data_path}")
all_normalized_frame = pandas.read_parquet(parquet_process_data_path)
display(all_normalized_frame)

Reading from parquet /data/parquet/processed_data.parquet


,subreddit,file_name,text,original_caption,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej
1,SFWNextDoorGirls,GgFEagO.jpg,Thoughts about my NYE party outfit?,a woman sitting on a table holding a pink flower,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,a woman sitting on a table holding a pink flower,False,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,False,9951b4f82caeb8ba2bd9f79f8d422450,1000d16
2,HotGirlNextDoor,nwa7hts2jb9a1.jpg,(IKTR),a woman in a bikini with a cell phone in her hand,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,a woman in a bikini with a cell phone in her hand,False,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,False,be8dd55e34216bec1e15e03fa296eacc,1000fg0
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,a collage of photos showing a man in a suit an...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n
...,...,...,...,...,...,...,...,...,...,...,...
8595,AmIhotAF,hgktc1on2b9a1.jpg,Good evening everyone ❤️,A girl in a red jacket is sitting on a red horse.,D:\data\images\AmIhotAF\thumbnail\hgktc1on2b9a...,a woman in a bikini with a cell phone in her hand,False,D:\data\images\AmIhotAF\hgktc1on2b9a1.jpg,False,9279353b1761d62e7f8ca587a4146abf,zzyerp
8596,greentext,bgho6WK.jpg,Anon does a little trolling,A man wearing a black shirt and black pants is...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi
8597,SFWRedheads,3rv0ql34bb9a1.jpg,Cute but Deadly 😋,A little girl in a pink dress is sitting on a ...,D:\data\images\SFWRedheads\thumbnail\3rv0ql34b...,NaN,False,D:\data\images\SFWRedheads\3rv0ql34bb9a1.jpg,False,ba89401054b09d3ff2b608705f8a7110,zzzg13
8598,CityPorn,Minquan_Bridge2017_TAIWAN.jpg,"Taipei, Taiwan",Error in captioning image,D:\data\images\CityPorn\thumbnail\Minquan_Brid...,NaN,False,D:\data\images\CityPorn\Minquan_Bridge2017_TAI...,False,3cac9b29f27d5fc26ddcec4e1641649f,zzzlyl


In [10]:
print("Filtering Subreddits with Images")
filtered_on_exist = all_normalized_frame[
	(all_normalized_frame["thumbnail_exists"] == True) &
	(all_normalized_frame["original_image_exists"] == True)
	]

total_images_display = filtered_on_exist.groupby("subreddit").size().reset_index(name="count")
display(total_images_display.sort_values("count", ascending=False))
print(f"Total Records {total_images_display['count'].sum()}")

Filtering Subreddits with Images


,subreddit,count
3,EarthPorn,929
7,SFWRedheads,883
2,CityPorn,846
4,Faces,742
6,SFWNextDoorGirls,538
9,greentext,530
11,sfwpetite,265
5,HotGirlNextDoor,262
0,AmIhotAF,208
10,selfies,95


Total Records 5401


In [11]:
print("Data With Known Images")
display(filtered_on_exist)

Data With Known Images


,subreddit,file_name,text,original_caption,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,a collage of photos showing a man in a suit an...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n
7,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,a beautiful young woman in a bikini posing for...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp
10,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,a woman in a bikini standing on a beach,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,a woman in a bikini standing on a beach,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd
...,...,...,...,...,...,...,...,...,...,...,...
8592,SFWRedheads,eu6z40qyya9a1.jpg,Starting the day with mushroom water in my mus...,A woman in a pink shirt and black pants is tal...,D:\data\images\SFWRedheads\thumbnail\eu6z40qyy...,a woman holding up a bottle of beer,True,D:\data\images\SFWRedheads\eu6z40qyya9a1.jpg,True,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq
8593,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,A girl in a pink dress is talking to another g...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,a young woman in a red dress is talking on the...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq
8594,CityPorn,dwu7i1dni99a1.jpg,"Ronda's bridge is brutal, Spain",A group of people are standing in front of a l...,D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...,a bridge over a body of water with a waterfall,True,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,True,02ae3631262b2de84b0b24c4275deae9,zzy49v
8596,greentext,bgho6WK.jpg,Anon does a little trolling,A man wearing a black shirt and black pants is...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi


In [12]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, GPT2TokenizerFast
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained(pretrained_model_name_or_path="D:\\models\\vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained(
	pretrained_model_name_or_path="D:\\models\\vit-gpt2-image-captioning")
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_model_name_or_path="D:\\models\\vit-gpt2-image-captioning")


def caption_image(image_path: str, index: int) -> str:
	try:
		device = torch.device(f"cpu")

		model.to(device)

		img = Image.open(image_path)
		if img.mode != 'RGB':
			img = img.convert(mode="RGB")

		pixel_values = feature_extractor(images=[img], return_tensors="pt").pixel_values

		pixel_values = pixel_values.to(device)

		max_length = 50
		num_beams = 1

		# get model prediction
		output_ids = model.generate(pixel_values, num_beams=num_beams, max_length=max_length)

		# decode the generated prediction
		predictions = tokenizer.decode(output_ids[0], skip_special_tokens=True)
		return predictions

	except Exception as e:
		print(f":: Error in caption_image: {e}")
		return "bruh"
	finally:
		pass

In [13]:
def wrapper_for_blip_thumbnail(row: object) -> str:
	bruh = "bruh"
	try:
		caption = row["thumbnail_caption"]
		image_path = row["thumbnail_path"]
		foo = row.__dict__['_name']  # Fucking silly
		row_index = int(foo)
		progress.update()

		if caption and len(caption) > 5:
			return caption
		else:
			return caption_image(image_path, row_index)
	except Exception as e:
		print(e)
		return bruh

In [14]:
def wrapper_for_blip(row: object) -> str:
	bruh = "bruh"
	try:
		caption = row["original_caption"]
		image_path = row["original_image"]
		foo = row.__dict__['_name']  # Fucking silly
		row_index = int(foo)
		progress.update()

		if caption and len(caption) > 5:
			return caption
		else:
			return caption_image(image_path, row_index)
	except Exception as e:
		print(e)
		return bruh

In [15]:
temp = filtered_on_exist.copy()
ddf = dd.from_pandas(temp, npartitions=12)

In [16]:
progress: tqdm = InnerProgressBar(len(temp), "Captioning-Primary-Images")
print(f"Total Images: {progress.total}")
temp['new_column'] = ddf.apply(lambda x: wrapper_for_blip(x), axis=1, meta=('str', object)).compute()

Captioning-Primary-Images: 0it [00:00, ?it/s]

Total Images: 5401


global:   0%|          | 0/12 [00:00<?, ?it/s]

Captioning-Primary-Images:  87%|████████▋ | 4718/5401 [00:00<00:00, 4695.24it/s]

In [17]:
print("=== Before Drop ===")
display(temp)
temp.rename(columns={"original_caption": "original_caption_old"}, inplace=True)

temp.rename(columns={"new_column": "original_caption"}, inplace=True)
temp.drop(columns=["original_caption_old"], inplace=True)
print("=== After Drop ===")
display(temp)

=== Before Drop ===


,subreddit,file_name,text,original_caption,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,new_column
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,a collage of photos showing a man in a suit an...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...
7,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,a beautiful young woman in a bikini posing for...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,a beautiful young woman in a bikini posing for...
10,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,a woman in a bikini standing on a beach,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,a woman in a bikini standing on a beach,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd,a woman in a bikini standing on a beach
...,...,...,...,...,...,...,...,...,...,...,...,...
8592,SFWRedheads,eu6z40qyya9a1.jpg,Starting the day with mushroom water in my mus...,A woman in a pink shirt and black pants is tal...,D:\data\images\SFWRedheads\thumbnail\eu6z40qyy...,a woman holding up a bottle of beer,True,D:\data\images\SFWRedheads\eu6z40qyya9a1.jpg,True,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,A woman in a pink shirt and black pants is tal...
8593,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,A girl in a pink dress is talking to another g...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,a young woman in a red dress is talking on the...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq,A girl in a pink dress is talking to another g...
8594,CityPorn,dwu7i1dni99a1.jpg,"Ronda's bridge is brutal, Spain",A group of people are standing in front of a l...,D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...,a bridge over a body of water with a waterfall,True,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,True,02ae3631262b2de84b0b24c4275deae9,zzy49v,A group of people are standing in front of a l...
8596,greentext,bgho6WK.jpg,Anon does a little trolling,A man wearing a black shirt and black pants is...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...


=== After Drop ===


,subreddit,file_name,text,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...
7,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,a beautiful young woman in a bikini posing for...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,a beautiful young woman in a bikini posing for...
10,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,a woman in a bikini standing on a beach,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd,a woman in a bikini standing on a beach
...,...,...,...,...,...,...,...,...,...,...,...
8592,SFWRedheads,eu6z40qyya9a1.jpg,Starting the day with mushroom water in my mus...,D:\data\images\SFWRedheads\thumbnail\eu6z40qyy...,a woman holding up a bottle of beer,True,D:\data\images\SFWRedheads\eu6z40qyya9a1.jpg,True,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,A woman in a pink shirt and black pants is tal...
8593,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,a young woman in a red dress is talking on the...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq,A girl in a pink dress is talking to another g...
8594,CityPorn,dwu7i1dni99a1.jpg,"Ronda's bridge is brutal, Spain",D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...,a bridge over a body of water with a waterfall,True,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,True,02ae3631262b2de84b0b24c4275deae9,zzy49v,A group of people are standing in front of a l...
8596,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...


In [18]:
print(f"Saving to parquet {parquet_process_data_path}")
temp.to_parquet(parquet_process_data_path)
del temp

Saving to parquet /data/parquet/processed_data.parquet


In [19]:
print(f"Reading from parquet {parquet_process_data_path} with Updated Primary Captions")
processed_with_captions = pandas.read_parquet(parquet_process_data_path)
display(processed_with_captions)

Reading from parquet /data/parquet/processed_data.parquet with Updated Primary Captions


,subreddit,file_name,text,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...
7,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,a beautiful young woman in a bikini posing for...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,a beautiful young woman in a bikini posing for...
10,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,a woman in a bikini standing on a beach,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd,a woman in a bikini standing on a beach
...,...,...,...,...,...,...,...,...,...,...,...
8592,SFWRedheads,eu6z40qyya9a1.jpg,Starting the day with mushroom water in my mus...,D:\data\images\SFWRedheads\thumbnail\eu6z40qyy...,a woman holding up a bottle of beer,True,D:\data\images\SFWRedheads\eu6z40qyya9a1.jpg,True,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,A woman in a pink shirt and black pants is tal...
8593,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,a young woman in a red dress is talking on the...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq,A girl in a pink dress is talking to another g...
8594,CityPorn,dwu7i1dni99a1.jpg,"Ronda's bridge is brutal, Spain",D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...,a bridge over a body of water with a waterfall,True,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,True,02ae3631262b2de84b0b24c4275deae9,zzy49v,A group of people are standing in front of a l...
8596,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...


In [20]:
temp = processed_with_captions.copy()
ddf = dd.from_pandas(temp, npartitions=12)

In [21]:
progress = InnerProgressBar(len(temp), "Captioning-Thumbnails-Images")
print(f"Total Images: {progress.total}")
temp['new_column'] = ddf.apply(lambda x: wrapper_for_blip_thumbnail(x), axis=1, meta=('str', object)).compute()


Captioning-Primary-Images: 100%|██████████| 5401/5401 [00:01<00:00, 3260.63it/s]

Total Images: 5401


global:   0%|          | 0/12 [00:00<?, ?it/s]


Captioning-Thumbnails-Images:  92%|█████████▏| 4952/5401 [00:00<00:00, 12218.63it/s]

In [22]:
print("=== Before Drop ===")
display(temp)
temp.rename(columns={"thumbnail_caption": "thumbnail_caption_old"}, inplace=True)
temp.rename(columns={"new_column": "thumbnail_caption"}, inplace=True)
temp.drop(columns=["thumbnail_caption_old"], inplace=True)
print("=== After Drop ===")
display(temp)

,subreddit,file_name,text,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,new_column
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...
7,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,a beautiful young woman in a bikini posing for...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,a beautiful young woman in a bikini posing for...,a beautiful young woman in a bikini posing for...
10,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,a woman in a bikini standing on a beach,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd,a woman in a bikini standing on a beach,a woman in a bikini standing on a beach
...,...,...,...,...,...,...,...,...,...,...,...,...
8592,SFWRedheads,eu6z40qyya9a1.jpg,Starting the day with mushroom water in my mus...,D:\data\images\SFWRedheads\thumbnail\eu6z40qyy...,a woman holding up a bottle of beer,True,D:\data\images\SFWRedheads\eu6z40qyya9a1.jpg,True,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,A woman in a pink shirt and black pants is tal...,a woman holding up a bottle of beer
8593,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,a young woman in a red dress is talking on the...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq,A girl in a pink dress is talking to another g...,a young woman in a red dress is talking on the...
8594,CityPorn,dwu7i1dni99a1.jpg,"Ronda's bridge is brutal, Spain",D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...,a bridge over a body of water with a waterfall,True,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,True,02ae3631262b2de84b0b24c4275deae9,zzy49v,A group of people are standing in front of a l...,a bridge over a body of water with a waterfall
8596,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...,a series of photos showing a man in a suit


,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...
7,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,a beautiful young woman in a bikini posing for...,a beautiful young woman in a bikini posing for...
10,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd,a woman in a bikini standing on a beach,a woman in a bikini standing on a beach
...,...,...,...,...,...,...,...,...,...,...,...
8592,SFWRedheads,eu6z40qyya9a1.jpg,Starting the day with mushroom water in my mus...,D:\data\images\SFWRedheads\thumbnail\eu6z40qyy...,True,D:\data\images\SFWRedheads\eu6z40qyya9a1.jpg,True,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,A woman in a pink shirt and black pants is tal...,a woman holding up a bottle of beer
8593,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq,A girl in a pink dress is talking to another g...,a young woman in a red dress is talking on the...
8594,CityPorn,dwu7i1dni99a1.jpg,"Ronda's bridge is brutal, Spain",D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...,True,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,True,02ae3631262b2de84b0b24c4275deae9,zzy49v,A group of people are standing in front of a l...,a bridge over a body of water with a waterfall
8596,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...,a series of photos showing a man in a suit


In [23]:
print(f"Saving to parquet {parquet_process_data_path} with Updated Thumbnail Captions")
temp.to_parquet(parquet_process_data_path)
del temp

Saving to parquet /data/parquet/processed_data.parquet with Updated Thumbnail Captions


,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...
7,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,a beautiful young woman in a bikini posing for...,a beautiful young woman in a bikini posing for...
10,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd,a woman in a bikini standing on a beach,a woman in a bikini standing on a beach
...,...,...,...,...,...,...,...,...,...,...,...
8592,SFWRedheads,eu6z40qyya9a1.jpg,Starting the day with mushroom water in my mus...,D:\data\images\SFWRedheads\thumbnail\eu6z40qyy...,True,D:\data\images\SFWRedheads\eu6z40qyya9a1.jpg,True,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,A woman in a pink shirt and black pants is tal...,a woman holding up a bottle of beer
8593,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq,A girl in a pink dress is talking to another g...,a young woman in a red dress is talking on the...
8594,CityPorn,dwu7i1dni99a1.jpg,"Ronda's bridge is brutal, Spain",D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...,True,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,True,02ae3631262b2de84b0b24c4275deae9,zzy49v,A group of people are standing in front of a l...,a bridge over a body of water with a waterfall
8596,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...,a series of photos showing a man in a suit


In [24]:
print(f"Reading from parquet {parquet_process_data_path} with Updated Thumbnail Captions")
processed_with_captions_more = pandas.read_parquet(parquet_process_data_path)
display(processed_with_captions_more)

Reading from parquet /data/parquet/processed_data.parquet with Updated Thumbnail Captions


,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...
7,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,a beautiful young woman in a bikini posing for...,a beautiful young woman in a bikini posing for...
10,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd,a woman in a bikini standing on a beach,a woman in a bikini standing on a beach
...,...,...,...,...,...,...,...,...,...,...,...
8592,SFWRedheads,eu6z40qyya9a1.jpg,Starting the day with mushroom water in my mus...,D:\data\images\SFWRedheads\thumbnail\eu6z40qyy...,True,D:\data\images\SFWRedheads\eu6z40qyya9a1.jpg,True,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,A woman in a pink shirt and black pants is tal...,a woman holding up a bottle of beer
8593,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq,A girl in a pink dress is talking to another g...,a young woman in a red dress is talking on the...
8594,CityPorn,dwu7i1dni99a1.jpg,"Ronda's bridge is brutal, Spain",D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...,True,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,True,02ae3631262b2de84b0b24c4275deae9,zzy49v,A group of people are standing in front of a l...,a bridge over a body of water with a waterfall
8596,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...,a series of photos showing a man in a suit


In [25]:
print("Filtering Subreddits with Images By original_caption")
filtered_captions = processed_with_captions_more[
	(processed_with_captions_more["original_caption"] != "bruh") &
	(~processed_with_captions_more["original_caption"].isna() | ~ processed_with_captions_more[
		"original_caption"].isnull())
	]

filtered_captions_display = filtered_captions.groupby("subreddit").size().reset_index(name="count")

display(filtered_captions_display.sort_values("count", ascending=False))
print(f"Total Records {filtered_captions_display['count'].sum()}")

Filtering Subreddits with Images By original_caption


,subreddit,count
3,EarthPorn,929
7,SFWRedheads,883
2,CityPorn,846
4,Faces,742
6,SFWNextDoorGirls,538
9,greentext,530
11,sfwpetite,265
5,HotGirlNextDoor,262
0,AmIhotAF,208
10,selfies,95


Total Records 5401


In [27]:
print("Filtering Subreddits with Images By thumbnail_caption")
filtered_captions_by_thumbnail = filtered_captions[
	(processed_with_captions_more["thumbnail_caption"] != "bruh") &
	(~processed_with_captions_more["thumbnail_caption"].isna() | ~ processed_with_captions_more[
		"thumbnail_caption"].isnull())
	]

filtered_captions_by_thumbnail_display = filtered_captions_by_thumbnail.groupby("subreddit").size().reset_index(
	name="count")
display(filtered_captions_by_thumbnail_display.sort_values("count", ascending=False))
print(f"Total Records {filtered_captions_by_thumbnail_display['count'].sum()}")

Filtering Subreddits with Images By thumbnail_caption


,subreddit,count
3,EarthPorn,929
7,SFWRedheads,883
2,CityPorn,846
4,Faces,742
6,SFWNextDoorGirls,538
9,greentext,530
11,sfwpetite,265
5,HotGirlNextDoor,262
0,AmIhotAF,208
10,selfies,95


Total Records 5401


In [ ]:
print("Updating Cloud Storage With Filtered Captioned Images")
def update_cloud(row):
	try:
		_table_adapter = TableAdapter()
		partition_key = "training"
		row_key = row["id"]
		caption = row["original_caption"]
		updated_caption = row["thumbnail_caption"]
		entity = _table_adapter.get_entity("training", partition_key, row_key)
		entity["caption"] = caption
		entity["updated_caption"] = updated_caption
		_table_adapter.upsert_entity_to_table("training", entity)
		progress.update()
		return "bruh"
	except Exception as e:
		print(e)
		return "bruh"

In [ ]:
progress = InnerProgressBar(len(processed_with_captions_more), "upload to cloud")
temp = processed_with_captions_more.copy()
ddf = dd.from_pandas(temp, npartitions=12)

In [ ]:
ddf.apply(lambda x: update_cloud(x), axis=1, meta=('str', object)).compute()

In [ ]:
print("Process Complete: Items Updated")